In [1]:
import os
os.chdir('../')

from extract import open_server_users
from extract import open_server_engage
from extract import open_server_payment
from extract import open_server_battle
from extract import open_server_resource
from extract import open_server_troop
from extract import server_open_nd
from extract import open_server_goldfarmer

from transform import goldfarmer_transform
from pipeline import feature_generation

import TASKS

from data_utils import run_aggregation_tasks,asfreq_by_group

import numpy as np
from config import ETLConfig
from functools import reduce

In [2]:
task = dict(game = 'aoz',server_ids = [234,235,236],
            offset = ETLConfig.OFFSET,days =ETLConfig.OPEN_SERVER_DAYS)

task['server_open'] = server_open_nd(**task)

# resource = open_server_resource(**task, fields = ETLConfig.RESOURCE_FIELDS)

In [3]:
feature_generation(task, ETLConfig)

GoldFarmerRob 236: 100%|████████████████████████████████████| 3/3 [00:38<00:00, 12.79s/it]


register_count  language_1_count_user_id  \
from_server day_id                                             
234         1                 6730                     120.0   
            2                48518                    1861.0   
            3                14478                     942.0   
            4                  389                     214.0   
            5                  135                      13.0   
            6                   91                      12.0   
235         1                34540                    1123.0   
            2                36972                    1978.0   
            3                  289                      17.0   
            4                  168                      11.0   
            5                  133                       7.0   
            6                   90                       3.0   
236         1                13670                     243.0   
            2                49517                    1906.0   
            3                11893                     602.0   
            4                  173                       8.0   
            5                  110                       6.0   
            6                   81                       6.0   

                    language_2_count_user_id  language_3_count_user_id  \
from_server day_id                                                       
234         1                           25.0                    2706.0   
            2                          426.0                   15737.0   
            3                          140.0                    4758.0   
            4                            1.0                      77.0   
            5                            0.0                      36.0   
            6                            2.0                      30.0   
235         1                          286.0                   11327.0   
            2                          365.0                   11376.0   
            3                            4.0                      82.0   
            4                            9.0                      35.0   
            5                            3.0                      27.0   
            6                            0.0                      22.0   
236         1                           72.0                    5278.0   
            2                          407.0                   16151.0   
            3                          113.0                    4150.0   
            4                            0.0                      61.0   
            5                            0.0                      42.0   
            6                            0.0                      25.0   

                    language_5_count_user_id  language_6_count_user_id  \
from_server day_id                                                       
234         1                          199.0                     134.0   
            2                         2814.0                    2003.0   
            3                         1228.0                     907.0   
            4                           12.0                       4.0   
            5                           14.0                       3.0   
            6                            5.0                       1.0   
235         1                         1715.0                    1465.0   
            2                         2849.0                    1360.0   
            3                            5.0                      15.0   
            4                           17.0                       6.0   
            5                           10.0                      12.0   
            6                            7.0                       3.0   
236         1                          231.0                     154.0   
            2                         1935.0                    1395.0   
            3                          594.0                     370.0   


In [2]:
task = dict(game = 'aoz',server_ids = [234,235,236],
            offset = ETLConfig.OFFSET,days =ETLConfig.OPEN_SERVER_DAYS)

task['server_open'] = server_open_nd(**task)

user = open_server_users(**task, fields = ETLConfig.USER_INFO_FIELDS)

engage = open_server_engage(**task, fields = ETLConfig.ENGAGE_FIELDS)

payment = open_server_payment(**task, fields = ETLConfig.PAYMENT_FIELDS)
payment =run_aggregation_tasks(payment, TASKS.PAYMENT_TO_DAILY,merge= True).reset_index()

battle = open_server_battle(**task, fields = ETLConfig.BATTLE_FIELDS)

resource = open_server_resource(**task, fields = ETLConfig.RESOURCE_FIELDS)

troop = open_server_troop(**task,fields = ETLConfig.TROOP_FIELDS)

goldfarmer = open_server_goldfarmer(**task, fields = ETLConfig.GOLD_FARMER_FIELDS)

goldfarmer = goldfarmer_transform(goldfarmer, ETLConfig)

GoldFarmerRob 236: 100%|████████████████████████████████████| 3/3 [01:23<00:00, 27.80s/it]


In [3]:
import pandas as pd
from datetime import timedelta

In [4]:
# def asfreq_by_group(df, group_col = 'user_id', date_col = 'date', fixed_last_day = None):

#     # get minimm date of a user, and maximum
#     first = df.groupby('user_id')['date'].min().reset_index()

#     if not fixed_last_day:
#         last = df.groupby('user_id')['date'].max().reset_index()

#     # if you want to end at specific day
#     else:
#         last = first.copy()

#         if isinstance(fixed_last_day, str):
#             try:
#                 fixed_last_day = datetime.strptime(fixed_last_day,'%Y-%m-%d')
#             except Exception as e:
#                 print('Bad datetime format for `fixed_last_day`')
#                 raise ValueError('Bad datetime format for `fixed_last_day`')

#         last['date'] = fixed_last_day


#     first = first.rename(columns = {'date':'first_date'})
#     last = last.rename(columns = {'date':'last_date'})

#     user_dates = first.merge(last, on = ['user_id'], how = 'left')


#     user_dates = user_dates.loc[user_dates.first_date <= user_dates.last_date,:]

    
#     # why use a loop: get faster speed
#     results = []
#     for user, s, e in user_dates.values:
#         cur_date = s
#         while s<= e:
#             results.append([user,s])
#             s += timedelta(days = 1)
#     filled = pd.DataFrame(results, columns = ['user_id','date'])

#     filled = filled.merge(df, on = ['user_id','date'], how = 'left').ffill()

#     return filled

In [5]:
def gen_city_level_features(resource):
    resource['city_level'] = resource['city_level'].astype(int)
    level_features_all_server = []
    
    # Loop through each server
    for server_id in resource.from_server.unique():
        
        # select current server data
        sub_resource = resource[resource.from_server == server_id].copy()
        sub_resource['date'] = pd.to_datetime(sub_resource['date'])
        sub_resource = sub_resource[['user_id','date','day_id','city_level']]
        
        # keep a record of date mapping
        mapping = sub_resource[['date','day_id']].drop_duplicates().values
        mapping = dict(zip(mapping[:,0], mapping[:,1]))
        
        # as freq operation
        level_features = asfreq_by_group(sub_resource)
        level_features['day_id'] = level_features['date'].replace(mapping)
        level_features['from_server'] = server_id
        level_features_all_server.append(level_features)
    city_level = pd.concat(level_features_all_server, sort = False)
    city_level['city_level'] = city_level['city_level'].round().astype(int)
    
    return run_aggregation_tasks(city_level, TASKS.CITYLEVEL_TASK, merge = True)

In [6]:
gen_city_level_features(resource)

city_level_0_count_user_id  city_level_1_count_user_id  \
from_server day_id                                                           
234         1                           2538.0                       856.0   
            2                          16520.0                      6728.0   
            3                           5597.0                      2668.0   
            4                            785.0                       562.0   
            5                            392.0                       296.0   
            6                            112.0                       136.0   
235         1                          11932.0                      4638.0   
            2                          13005.0                      5512.0   
            3                           1196.0                       910.0   
            4                            657.0                       481.0   
            5                            348.0                       278.0   
            6                            107.0                       152.0   
236         1                           5168.0                      1911.0   
            2                          17467.0                      7102.0   
            3                           5114.0                      2672.0   
            4                            894.0                       685.0   
            5                            487.0                       344.0   
            6                            153.0                       154.0   

                    city_level_2_count_user_id  city_level_3_count_user_id  \
from_server day_id                                                           
234         1                           1393.0                       813.0   
            2                           9788.0                      5935.0   
            3                           4344.0                      3084.0   
            4                            928.0                       913.0   
            5                            517.0                       483.0   
            6                            237.0                       201.0   
235         1                           6827.0                      4056.0   
            2                           8330.0                      5310.0   
            3                           1584.0                      1387.0   
            4                            834.0                       750.0   
            5                            480.0                       401.0   
            6                            204.0                       160.0   
236         1                           2735.0                      1609.0   
            2                          10206.0                      6401.0   
            3                           4132.0                      2980.0   
            4                           1131.0                      1059.0   
            5                            624.0                       530.0   
            6                            260.0                       233.0   

                    city_level_4_count_user_id  city_level_5_count_user_id  \
from_server day_id                                                           
234         1                            647.0                       303.0   
            2                           5089.0                      4262.0   
            3                           3366.0                      4205.0   
            4                           1462.0                      2439.0   
            5                            712.0                      1470.0   
            6                            330.0                       776.0   
235         1                           3276.0                      2543.0   
            2                           5137.0                      5113.0   
            3                           1938.0                      3121.0   
            4                           

In [89]:
city_level = gen_city_level_features(resource)

In [90]:
task = ((['from_server','day_id'],['city_level'],['user_id'],['count'],0),)

In [93]:
city_level['city_level'] = city_level['city_level'].round().astype(int)

In [94]:
run_aggregation_tasks(city_level, task, merge = True)

city_level_0_count_user_id  city_level_1_count_user_id  \
from_server day_id                                                           
234         1                           2538.0                       856.0   
            2                          16520.0                      6728.0   
            3                           5597.0                      2668.0   
            4                            785.0                       562.0   
            5                            392.0                       296.0   
            6                            112.0                       136.0   
235         1                          11932.0                      4638.0   
            2                          13005.0                      5512.0   
            3                           1196.0                       910.0   
            4                            657.0                       481.0   
            5                            348.0                       278.0   
            6                            107.0                       152.0   
236         1                           5168.0                      1911.0   
            2                          17467.0                      7102.0   
            3                           5114.0                      2672.0   
            4                            894.0                       685.0   
            5                            487.0                       344.0   
            6                            153.0                       154.0   

                    city_level_2_count_user_id  city_level_3_count_user_id  \
from_server day_id                                                           
234         1                           1393.0                       813.0   
            2                           9788.0                      5935.0   
            3                           4344.0                      3084.0   
            4                            928.0                       913.0   
            5                            517.0                       483.0   
            6                            237.0                       201.0   
235         1                           6827.0                      4056.0   
            2                           8330.0                      5310.0   
            3                           1584.0                      1387.0   
            4                            834.0                       750.0   
            5                            480.0                       401.0   
            6                            204.0                       160.0   
236         1                           2735.0                      1609.0   
            2                          10206.0                      6401.0   
            3                           4132.0                      2980.0   
            4                           1131.0                      1059.0   
            5                            624.0                       530.0   
            6                            260.0                       233.0   

                    city_level_4_count_user_id  city_level_5_count_user_id  \
from_server day_id                                                           
234         1                            647.0                       303.0   
            2                           5089.0                      4262.0   
            3                           3366.0                      4205.0   
            4                           1462.0                      2439.0   
            5                            712.0                      1470.0   
            6                            330.0                       776.0   
235         1                           3276.0                      2543.0   
            2                           5137.0                      5113.0   
            3                           1938.0                      3121.0   
            4                           

In [5]:
from camel_decorators import multiserver

In [6]:
user_features = run_aggregation_tasks(user, TASKS.USER_TASKS,merge = True)
user_features = user_features.rename(columns = {'user_id_count':'register_count'})

engage_features = run_aggregation_tasks(engage, TASKS.ENGAGE_TASKS, merge = True)

battle_features = run_aggregation_tasks(battle, TASKS.BATTLE_TASK, merge = True)
battle_features = battle_features.rename(columns = {'user_id_count':'battle_count'})

troop_features = run_aggregation_tasks(troop, TASKS.TROOP_TASK, merge = True)

resource_features = run_aggregation_tasks(resource, TASKS.RESOURCE_TASK,merge = True)

In [7]:
feature_list = [user_features, engage_features, battle_features, troop_features, resource_features]

In [8]:
reduce(lambda left, right: left.merge(right, left_index = True, right_index = True), feature_list)

register_count  language_1_count_user_id  \
from_server day_id                                             
234         1                 6730                     120.0   
            2                48518                    1861.0   
            3                14478                     942.0   
            4                  389                     214.0   
            5                  135                      13.0   
            6                   91                      12.0   
235         1                34540                    1123.0   
            2                36972                    1978.0   
            3                  289                      17.0   
            4                  168                      11.0   
            5                  133                       7.0   
            6                   90                       3.0   
236         1                13670                     243.0   
            2                49517                    1906.0   
            3                11893                     602.0   
            4                  173                       8.0   
            5                  110                       6.0   
            6                   81                       6.0   

                    language_2_count_user_id  language_3_count_user_id  \
from_server day_id                                                       
234         1                           25.0                    2706.0   
            2                          426.0                   15737.0   
            3                          140.0                    4758.0   
            4                            1.0                      77.0   
            5                            0.0                      36.0   
            6                            2.0                      30.0   
235         1                          287.0                   11326.0   
            2                          365.0                   11376.0   
            3                            4.0                      82.0   
            4                            9.0                      35.0   
            5                            3.0                      27.0   
            6                            0.0                      22.0   
236         1                           72.0                    5278.0   
            2                          407.0                   16151.0   
            3                          113.0                    4150.0   
            4                            0.0                      61.0   
            5                            0.0                      42.0   
            6                            0.0                      25.0   

                    language_5_count_user_id  language_6_count_user_id  \
from_server day_id                                                       
234         1                          199.0                     134.0   
            2                         2814.0                    2003.0   
            3                         1228.0                     907.0   
            4                           12.0                       4.0   
            5                           14.0                       3.0   
            6                            5.0                       1.0   
235         1                         1715.0                    1465.0   
            2                         2849.0                    1360.0   
            3                            5.0                      15.0   
            4                           17.0                       6.0   
            5                           10.0                      12.0   
            6                            7.0                       3.0   
236         1                          231.0                     154.0   
            2                         1935.0                    1395.0   
            3                          594.0                     370.0   


In [32]:
server_open_nd(days = 30, offset = 0, game = 'aoz')

,open_time,open_date,open_30_date,open_30_date_id,open_date_id
server_id,,,,,
0,2020-08-10,2020-08-10,2020-09-09,20200909,20200810
1,2018-09-20,2018-09-20,2018-10-20,20181020,20180920
2,2018-10-25,2018-10-25,2018-11-24,20181124,20181025
3,2018-11-16,2018-11-16,2018-12-16,20181216,20181116
4,2018-11-18,2018-11-18,2018-12-18,20181218,20181118
...,...,...,...,...,...
354,2020-11-30,2020-11-30,2020-12-30,20201230,20201130
355,2020-12-02,2020-12-02,2021-01-01,20210101,20201202
356,2020-12-04,2020-12-04,2021-01-03,20210103,20201204


In [ ]:
from dbx import Dbtools
from camel_utils_x.camel_sql import SQLBase
from camel_queries.datafile_config import all_paths, show_catalog
from data_utils import with_day_id
from .utils import rename_aggr, server_open_nd

from datetime import datetime, timedelta
import pandas as pd

import logging
from tqdm import tqdm

@with_day_id()
def open_server_payment(server_ids = [123,234], days = 10, server_open = None,
                    fields = None, game = 'aoz', offset = 0, **kwargs):

    tool = Dbtools.initialize('all',game)

    # default parameters
    if isinstance(server_ids, int):
        server_ids = [server_ids]

    if server_open is None:
        server_open =server_open_nd(game = game,days = days, offset = offset)

    if fields is None:
        fields = ['user_id','create_time','currency_ammount']

    # table
    table = 'payment_record'

    # parsers
    parsers = {'create_time':'stamp13'}

    alias = {'currency_ammount':'amount'}

    dfs = []
    pbar = tqdm(server_ids, desc = 'Payment: ')
    for server_id in pbar:
        pbar.set_description('Payment {}'.format(server_id))
        try:
            before_date = server_open.loc[server_id,f'open_{days}_date']
            after_date = server_open.loc[server_id, 'open_time']

            sql = SQLBase(fields = fields, table = table,
                          parsers = parsers, alias = alias)\
                        .stamp13before('create_time',before_date)\
                        .stamp13after('create_time',after_date).make()

            tool = Dbtools.initialize('all',game)
            conn = tool.get_connection(**tool.get_conn_info(server_id,'gs'))
            df = pd.read_sql_query(sql, conn)
            df['from_server'] = server_id
            dfs.append(df)
        except Exception as e:
            logging.error('Unable to get payment on server {}'.format(server_id))
            print(e)

    dfs = pd.concat(dfs, sort = False)
    dfs.columns = rename_aggr(dfs.columns, aggr = 'from_unixtime')
    # dfs.rename(columns = {'create_time':'date'}, inplace = True)
    dfs['date'] = pd.to_datetime(dfs.create_time.dt.date)


In [68]:
import os 
os.getcwd()

'C:\\Users\\wanghuan\\desktop\\camel_open_server'

In [77]:
import pandas as pd

In [70]:
from camel_utils_x.camel_sql import SQLBase
from extract.utils import rename_aggr, server_open_nd

In [71]:



server_id = 233
days = 30

In [72]:
server_open = server_open_nd(game = 'aoz', days = 30, offset = 0)

In [73]:
from dbx import Dbtools

In [184]:
@multiserver
def payment_nd(game = 'aoz', server_id = 233,days = 30,tool = None):
    global server_open
    fields = ['currency_ammount']
    table = 'payment_record'
    parsers = { 'currency_ammount':'sum'}
    alias = ['amount']
    before_date = server_open.loc[server_id,f'open_{days}_date']

    sql = SQLBase(fields = fields, table = table, 
                  parsers = parsers, 
                  alias = alias).\
        stamp13beforeq('currency_ammount', before_date).make()
    
    conn = tool.get_connection(**tool.get_conn_info(server_id,'gs'))
    df = pd.read_sql_query(sql,conn)
    return df

In [185]:
tool = Dbtools.initialize('all','aoz')

In [186]:
payment_nd(game = 'aoz', server_id = [1,2,3,4,5,6,7], tool = None)

,amount,from_server
0,3.880170e+06,1
0,4.780392e+06,2
0,2.602879e+05,3
0,1.444762e+05,4
0,3.816812e+05,5
0,1.429232e+05,6
0,4.838314e+05,7


In [36]:
user_type_col = [col for col in user_features.columns if col.startswith('user_type')]

In [39]:
utype = user_features[user_type_col].reset_index()

In [34]:
utype.reset_index().groupby('from_server')

In [44]:
utype.drop('day_id', axis = 1).groupby('from_server').cumsum()

,user_type_cur_dummy_account_count_user_id,user_type_cur_init_user_count_user_id,user_type_pure_roller_count_user_id,user_type_sandbox_count_user_id
0,3.0,6579.0,148.0,0.0
1,118.0,54715.0,415.0,0.0
2,268.0,68924.0,534.0,0.0
3,339.0,69195.0,581.0,0.0
4,390.0,69239.0,621.0,0.0
5,428.0,69263.0,650.0,0.0
6,45.0,34210.0,285.0,0.0
7,191.0,70826.0,495.0,0.0
8,281.0,70952.0,568.0,0.0
9,341.0,71005.0,623.0,0.0


In [45]:
from bokeh.plotting import figure

In [23]:
from dbx import Dbtools

In [25]:
tool = Dbtools.initialize('all','ody')

In [27]:
conn = tool.get_connection(**tool.get_conn_info(2,'gs'))

In [29]:
pd.read_sql_query('select * from user_info limit 10', conn).columns

Index(['id', 'name', 'sign', 'create_time', 'token', 'space_station_id',
       'alliance_id', 'sys_icon', 'custom_icon', 'language',
       'register_channel', 'login_channel', 'country_code', 'novice_protect',
       'bind_channel', 'server_id', 'device_id', 'device_model',
       'client_version', 'os_version', 'ip', 'current_device_id',
       'sub_channel', 'last_login_time', 'last_sync_time', 'ip_country',
       'register_ip_country', 'register_ip', 'agree_gdpr',
       'merge_from_server'],
      dtype='object')

In [31]:
pd.read_sql_query('select * from operate_res_log limit 10', conn).columns

Index(['id', 'user_id', 'uid', 'aid', 'date_id', 'last_time', 'city_level',
       'mojo', 'mojo_add', 'mojo_use', 'steel', 'steel_add', 'steel_use',
       'tritium', 'tritium_add', 'tritium_use', 'mythrill', 'mythrill_add',
       'mythrill_use', 'antimatter', 'antimatter_add', 'antimatter_use',
       'darkmatter', 'darkmatter_add', 'darkmatter_use'],
      dtype='object')